In [6]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import jieba
import jieba.analyse


In [9]:
df = pd.read_csv('movie.csv')
df=df.drop(labels=['Unnamed: 0'],axis='columns')
# df

topcut = []
for d in df['簡介']:
    top = jieba.analyse.extract_tags(d,topK=10)
    topcut.append(top)
topcut
# df['key']=df['中文名稱'].astype(str)+df['類型'].astype(str)+df['簡介']

# df.to_numpy()




[['電影', '這部', '小小', '動畫', '超級', '格魯', '導演', '克里斯', '系列', '照明'],
 ['蝙蝠', '一個', '布魯斯', '韋恩', '並從', '中檢視', '靈魂', '伯派', '森將', '中飾演'],
 ['奧斯卡', '雷托領', '治療', '實驗', '結果', '瑞德', '比斯', '宇宙', '疾病', '得主'],
 ['捍衛戰士', '任務', '1986', '湯姆克魯斯', '獨行俠', '50', '派拉蒙', '金斯基', '珍妮佛', '航空'],
 ['魔鬼', '未來', '童星', '保羅', '續集', '他們', '特曼', '系列', '路德', '正宗'],
 ['奧斯卡', '仇者', '聯盟', 'HBO', '提名', '本片', '講述', '一個', '資源', '人類'],
 ['龐德', '女郎', '007', '飾演', '電影', '一個', '一集', '詹姆斯', '全新', '角色'],
 ['猛毒', '蜘蛛', '票房', '2018', '累積', '8.5', '創下', '佳績', '將由安迪瑟', '執導'],
 ['摩加迪休', '大使', '導演', '內戰', '同行', '打造', '擊退', '強檔', '叢林', '奇航'],
 ['尚氣', '十環', '飾演', '父親', '再度', '漫威', '電影', '擴張', '動作', '傳奇'],
 ['冰路', '300', '英哩', '30', '小時', '危機', '駕駛', '卡車', '難隊', '救難'],
 ['殺手', '保鑣', '續集', '山繆', '保鏢', '導演', '票房', '原班', '必看', '億萬'],
 ['遊戲', '密室', '生存者', '頂尖', '逃脫', '紐約', '了將', '主辦', '方繩', '兩位'],
 ['蛇眼', '一個', '忍者', '特種', '部隊', '之戰', '嵐影', '氏族', '男爵夫人', '家族'],
 ['一個', '脫稿', '銀行', '職員', '蓋伊', '萊恩雷諾斯', '著無聊', '過著樸', '實無華', '發現'],
 ['關頭', '電影', '玩

In [3]:

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['key'])
feature_name = vectorizer.get_feature_names()


print (X)
print (feature_name)
print (X.toarray())

